In [5]:
# Magic comands
%reload_ext autoreload
%autoreload 2

# Libraries
import os, sys, webcolors
import numpy as np
import pandas as pd
import plotly.graph_objects as go

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

origin_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(origin_path, os.pardir))
os.chdir(parent_dir)

from app.nqueens import NQueensGeneticAlgorithm

In [6]:
def genetic_algorithm(
                    n = 8,
                    pop = 100,
                    gen = 50,
                    mut_perc = .05,
                    cross_perc = 0.75,
                    mut='swap_mutation',
                    cross='pmx',
                    select='tournament_selection',
                    ):
    
    nQueensGA = NQueensGeneticAlgorithm(population_size=pop, dimension=n)

    nQueensGA.run(
        generations=gen,
        mutation_probability=mut_perc,
        crossover_probability=cross_perc,
        mutation_operator=mut,
        crossover_operator=cross,
        selection_operator=select
    )

    return nQueensGA.report()

In [7]:
# testing function
ga = genetic_algorithm(
                                            n = 8,
                                            pop = 100,
                                            gen = 500,
                                            mut_perc = .02,
                                            cross_perc = .8,
                                            mut = "swap_mutation",
                                            cross = "single_cross",
                                            select = "tournament_selection"
                                            )
ga

{'generations': 500,
 'duration': 211,
 'best_fitness': 28,
 'best_fitness_percentage': 100.0,
 'best_representation': [2, 5, 1, 6, 0, 3, 7, 4],
 'worst_fitness': 26,
 'worst_representation': [2, 6, 1, 5, 0, 3, 7, 4],
 'mean_fitness': 27.97,
 'selection_operator': 'tournament_selection',
 'mutate_operator': 'swap_mutation',
 'crossover_operator': 'single_point_co'}

## __1. Optimization__

In [8]:
def best_operators_search(parameter: str, values: list):
    # dictionary with parameter names and default values 
    defaults = genetic_algorithm.__defaults__
    parameters = genetic_algorithm.__code__.co_varnames
    params = {param: val for param, val in zip(parameters, defaults)}
    # initializing variables
    best_fit = 0.0
    time_fit = 9e99
    fit_dict = {}
    time_dict = {}
    param_list = []

    # Perform grid search
    for i in values:
        # 100 iterations to generate statistically relevant results
        fit_list = []
        time_list = []
        for _ in range(100):
            # changing parameter according to input
            params[parameter] = i
            # Define the model with current hyperparameters
            ga = genetic_algorithm(**params)
            # gets restults
            fit = ga['best_fitness_percentage']
            time_it = ga['duration']
            # Check if current solution has better fitness than previous best 
            if fit > best_fit:
                best_fit = fit
                time_fit = time_it
                best_param = {parameter: i}
            # Check if the solution with fitness as good is faster than previous best
            if fit == best_fit and time_it < time_fit:
                time_fit = time_it
                best_param = {parameter: i}
            # Append results to get 100 iterations values
            fit_list.append(fit)
            time_list.append(time_it)
        # saving results per parameter
        param_list.append(best_param)
        fit_dict['Fit_' + i] = fit_list
        time_dict['Time_' + i] = time_list

    unique_best_params =list({tuple(d.items()): d for d in param_list}.values())

    # dataframe with results
    fit_dict.update(time_dict)
    df = pd.DataFrame(fit_dict)
        
    return unique_best_params, df

### __1.1 Mutation Module__

In [9]:
# seed for reproducibility
np.random.seed(0)
# Define the range of values for each hyperparameter
#xo = ['single_cross', 'cycle_cross', 'pmx', 'arithmetic_cross']
#select = ['tournament_selection', 'stochastic_universal_sampling']
mutation = ['swap_mutation', 'random_mutation', 'inversion_mutation']
#population = [100, 250, 500, 1000]
#generations = [100, 250, 500]
#cross_percentage = np.arange(0.8, 0.95, 0.05)
#mut_percentage = np.arange(0.01, 0.06, 0.01)

param_list, df = best_operators_search('mut', mutation)

In [41]:
def plot_values(data: pd.DataFrame, colors: list, layout: int, template:str, grid:  bool):
    # Filter columns starting with "Fit_"
    fit_columns = [col for col in data.columns if col.startswith('Fit_')]

    # Filter columns starting with "Time_"
    time_columns = [col for col in data.columns if col.startswith('Time_')]

    template_map = {'white':'plotly_white', 'dark': 'plotly_dark'}

    tick_font_map = {'white': webcolors.rgb_to_hex(webcolors.name_to_rgb('black')), 'dark': webcolors.rgb_to_hex(webcolors.name_to_rgb('white'))}

    # Layout styles
    layout_1 = {
        'xaxis_title': 'Iterations',
        'yaxis_title': 'Time (ms)',
        'template': template_map[template],
        'font': {'size': 14, 'family': 'Verdana'},
        'title': {'text': 'Time Comparison', 'font': {'size': 19, 'family': 'Verdana'}},
        'xaxis': {'showgrid': grid, 'ticks': 'outside', 'tickfont': {'color': tick_font_map[template]}},
        'yaxis': {'showgrid': grid, 'tickfont': {'color': tick_font_map[template]}}
        }
    
    layout_2 = {
        'title': 'Fitness Comparison',
        'xaxis_title': 'Iterations',
        'yaxis_title': 'Fitness',
        'template': template_map[template],
        'font': {'size': 14, 'family': 'Verdana'},
        'xaxis': {
            'showline': True,
            'showgrid': grid,
            'showticklabels': True,
            'linewidth': 1.5,
            'ticks': 'outside',
            'tickfont': {
                'family': 'Verdana',
                'size': 11,
                'color': tick_font_map[template]}},
        'yaxis': {
            'showgrid': grid,
            'zeroline': False,
            'showline': False,
            'showticklabels': True,
            'ticks': 'outside',
            'tickfont': {
                'family': 'Verdana',
                'size': 11,
                'color': tick_font_map[template]}}}
    
    layout_map = {1:layout_1, 2:layout_2}

    # Convert color names to RGB values
    rgb_colors = [webcolors.name_to_rgb(color) for color in colors]

    mode_size = 8
    line_size = 2

    # Create a subplot with two line plots
    fig = go.Figure()

    # Add line plots for "Fit_" columns
    i = 0
    for col in fit_columns:
        fig.add_trace(go.Scatter(x=data.index, y=data[col], name=col.replace('Fit_', ''),
                                line=dict(color=colors[i], width=line_size),
                                ))

        fig.add_trace(go.Scatter(
            x=[data.index[0], data.index[-1]],
            y=[data[col][0], data[col].iloc[-1]],
            mode='markers',
            marker=dict(color=colors[i], size=mode_size),
            name=''
        ))
        i += 1

    fig.update_layout(layout_map[layout])

    fig2 = go.Figure()

    # Add line plots for "Time_" columns
    i = 0
    for col in time_columns:
        fig2.add_trace(go.Scatter(x=data.index, y=data[col], name=col.replace('Time_', ''),
                                line=dict(color=colors[i], width=line_size),
                                ))

        fig2.add_trace(go.Scatter(
            x=[data.index[0], data.index[-1]],
            y=[data[col][0], data[col].iloc[-1]],
            mode='markers',
            marker=dict(color=colors[i], size=mode_size),
            name=''
        ))
        i += 1

    fig2.update_layout(layout_map[layout])

    # Show the plots
    fig.show()
    fig2.show()

In [42]:
plot_values(data = df, colors = ['black', 'gray', 'steelblue'], layout=2, template='dark', grid=True)

In [17]:
df['const'] = 'const'
df.groupby('const').mean().reset_index(drop=True).round(3)

,Fit_swap_mutation,Fit_random_mutation,Fit_inversion_mutation,Time_swap_mutation,Time_random_mutation,Time_inversion_mutation
0,99.929,99.929,99.964,20.88,21.37,21.19
